# Welcome to [FIXME]

This notebook provides an interactive interface that aids in the development of genetic algorithms, through enabling the user to select a predefined configuration for 1 of 4 common problem instances, as well as define a custom configuration, fitness evaluation function, and phenotype visualizion for their own problem. 

### Import dependencies

In [1]:
from jupyter_dash import JupyterDash
import numpy as np
from plotly import graph_objects as go
import app as app_file
import setup as setup_file
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from configuration import Config
from statistics import Statistics

### Setup the Dash environment

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

### Configure and save a genetic algorithm

In [4]:
configs = Config()
setup_applet = setup_file.create_app(configs)
setup_applet.run_server(mode='inline')

### Create a sample individual

In [6]:
sample_pop = configs.params['enc_type'].initialize()
sample_pop[0]

{'gene': array([10, 13, 27,  8, 21,  7,  3,  9, 12,  1, 20, 18, 15, 23, 16, 11, 24,
         2,  6,  4, 19, 25, 26, 14,  0,  5, 22, 28, 17]),
 'fitness': array([0])}

### Define a custom evaluation function

If you are defining your own algorithm configuration, your evaluation function should update all fitness values of an individual. Sample evaluation functions for both single and mult-objective problem instances can be found in ```evaluate.py```. If you are using a predefined configuration for a default problem instance, don't run this cell.



In [ ]:
def custom_eval(self, pop):
    pass

configs.params['eval_type'] = cust_eval

### Test the evaluation function

In [7]:
sample_pop = configs.params['eval_type'](configs, sample_pop)
sample_pop[0]

{'gene': array([10, 13, 27,  8, 21,  7,  3,  9, 12,  1, 20, 18, 15, 23, 16, 11, 24,
         2,  6,  4, 19, 25, 26, 14,  0,  5, 22, 28, 17]),
 'fitness': array([101620])}

### Define a custom phenotype visualization

If you are defining a custom algorithm configuration, your visualization function should define a phenotypic representation of an individual using the ```dash_core_components``` or ```dash_bootstrap_components``` libraries (see ```README.md```). Sample visualization functions for the predefined problems can be found in ```visualize.py```. If you are using a predefined configuration for a default problem instance, don't run this cell.


In [7]:
def cust_vis(ind):
    pass

configs.params['cust_vis'] = cust_vis

### Run the genetic algorithm

In [8]:
stats = Statistics(configs.params)
main_app = app_file.create_app(configs, stats)
main_app.run_server()

Dash app running on http://127.0.0.1:8050/


### Analyze the resulting population

In [ ]:
population = stats.posthoc['population']
population